In [6]:
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from Tools import tools

# importing data
# Read the 2D array back from the text file
dir_path = r"C:\Users\arpan\OneDrive\Documents\GRB\data\dp3-1d-5(-5_20)"
event_types = {1:'GRB',2:'TGF',3:'SGR',4:'SFLARE',5:'DISTPAR'}
event_counter = {'GRB':0,'TGF':0,'SGR':0,'SFLARE':0,'DISTPAR':0}
# Replace 'your_search_string' with the string you are looking for in file names
search_string = 'bn'
search_pattern = os.path.join(dir_path, f'*{search_string}*')
matching_files = glob.glob(search_pattern)

X = []
Y = []

for file in matching_files:
    event_type , event_name = file.split('\\')[-1].split('_')
    loaded_data = np.loadtxt(file, delimiter='\t').astype(np.int64)
    X.append(loaded_data)
    y = [0,0,0,0,0]
    for key,value in event_types.items():
        if value in file.split('\\')[-1]:
            # print(value, file)
            y[key-1] = 1
            Y.append(y)
            event_counter[value] += 1
            break
    
X = np.array(X)
Y = np.array(Y)
print(X.shape)
print(Y.shape)



(563, 32823)
(563, 5)


In [7]:
# Find the space taken by the array in bytes
array_size_bytes = X.nbytes 

print(f"Size of the array in bytes: {array_size_bytes} bytes")

Size of the array in bytes: 147834792 bytes


In [8]:
print(event_counter)

{'GRB': 127, 'TGF': 115, 'SGR': 110, 'SFLARE': 111, 'DISTPAR': 100}


In [9]:
print(X.shape)

(563, 32823)


In [10]:
# Split the train and the validation set for the fitting
from sklearn.model_selection import train_test_split
X_train = (X)
Y_train = (Y)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.2)#, random_state=2)
print("X_train shape",X_train.shape)
print("X_test shape",X_val.shape)
print("Y_train shape",Y_train.shape)
print("Y_test shape",Y_val.shape)

X_train shape (450, 32823)
X_test shape (113, 32823)
Y_train shape (450, 5)
Y_test shape (113, 5)


In [11]:
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.datasets import load_arrow_head
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

classifier = TimeSeriesForestClassifier()
classifier.fit(X_train, Y_train)
y_pred = classifier.predict(X_val)
accuracy_score(Y_val, y_pred)

c:\Users\arpan\miniconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [64]:
import tensorflow as tf
from keras import layers, models, activations
from keras.callbacks import EarlyStopping

# Build the CNN model
model = models.Sequential()
# model.add(layers.MaxPooling1D(pool_size=10, input_shape=(X_train.shape[1:]+tuple([1]))))
model.add(layers.Conv1D(filters=128, kernel_size = 64, input_shape=(X.shape[1:])+tuple([1])))
model.add(layers.Activation('selu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Conv1D(filters=128, kernel_size = 32))
model.add(layers.PReLU())
# model.add(layers.Conv1D(64,16,activation='mish'))
model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='selu'))
model.add(layers.Dense(5, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model
model.fit(X_train, Y_train, epochs=15, batch_size=16, validation_split=0.2,callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_train, Y_train)
print(f'\nTest accuracy on training data: {test_acc} \n \n')

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_val, Y_val, verbose=2)
print(f'Test accuracy on test data: {test_acc}')

Epoch 1/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 746s 32s/step - accuracy: 0.2343 - loss: 358.3183 - val_accuracy: 0.3222 - val_loss: 578.3525
Epoch 2/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 734s 32s/step - accuracy: 0.3440 - loss: 517.7601 - val_accuracy: 0.2000 - val_loss: 311.1645
Epoch 3/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 727s 32s/step - accuracy: 0.3060 - loss: 744.4716 - val_accuracy: 0.4333 - val_loss: 193.0031
Epoch 4/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 753s 33s/step - accuracy: 0.3108 - loss: 548.4081 - val_accuracy: 0.2778 - val_loss: 204.3175
Epoch 5/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 642s 28s/step - accuracy: 0.3596 - loss: 235.8717 - val_accuracy: 0.4778 - val_loss: 72.3197
Epoch 6/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 618s 27s/step - accuracy: 0.3012 - loss: 197.3609 - val_accuracy: 0.4111 - val_loss: 31.2765
Epoch 7/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 610s 27s/step - accuracy: 0.4125 - loss: 74.6168 - val_accuracy: 0.4000 - val_loss: 13.0928
Epoch 8/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 612s 27s/step - accuracy: 0.4593 - loss: 2

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from keras import layers, models, activations
from keras.callbacks import EarlyStopping

# Build the CNN model
model1 = models.Sequential()
# model.add(layers.MaxPooling1D(pool_size=10, input_shape=(X_train.shape[1:]+tuple([1]))))
model1.add(layers.Conv1D(filters=128, kernel_size = 64, input_shape=(X.shape[1:])+tuple([1])))
model1.add(layers.Activation('selu'))
model1.add(layers.BatchNormalization())
model1.add(layers.Dropout(0.5))
model1.add(layers.Conv1D(filters=64, kernel_size = 16))
model1.add(layers.PReLU())
# model.add(layers.Conv1D(64,16,activation='mish'))
model1.add(layers.Flatten())
model1.add(layers.Dense(64, activation='selu'))
model1.add(layers.Dense(5, activation='softmax'))

# Compile the model
model1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model
model1.fit(X_train, Y_train, epochs=15, batch_size=16, validation_split=0.2,callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_acc = model1.evaluate(X_train, Y_train)
print(f'\nTest accuracy on training data: {test_acc} \n \n')

# Evaluate the model on the test set
test_loss, test_acc = model1.evaluate(X_val, Y_val, verbose=2)
print(f'Test accuracy on test data: {test_acc}')

In [9]:
# saving the model
model.save('m01-02.keras')


In [6]:
import keras

model = keras.models.load_model('m31-01.keras')


c:\Users\arpan\miniconda3\lib\site-packages\keras\src\saving\saving_lib.py:394: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 11 variables whereas the saved optimizer has 20 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [7]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_train, Y_train)
print(f'\nTest accuracy on training data: {test_acc:.4f} \n \n')

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_val, Y_val, verbose=2)
print(f'Test accuracy on test data: {test_acc:.4f}')

15/15 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.6550 - loss: 29.8792

Test accuracy on training data: 0.6911 
 

4/4 - 7s - 2s/step - accuracy: 0.6991 - loss: 43.2309
Test accuracy on test data: 0.6991


In [8]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X, Y)
print(f'\nTest accuracy on entire data: {test_acc:.4f} \n \n')


18/18 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.5866 - loss: 29.6261

Test accuracy on entire data: 0.6927 
 



In [9]:
# Assuming 'model' is your Keras model
for i, layer in enumerate(model.layers):
    print(f"Layer {i+1} is a {layer}")


Layer 1 is a <Conv1D name=conv1d, built=True>
Layer 2 is a <PReLU name=p_re_lu, built=True>
Layer 3 is a <Conv1D name=conv1d_1, built=True>
Layer 4 is a <Flatten name=flatten, built=True>
Layer 5 is a <Dense name=dense, built=True>
Layer 6 is a <Dense name=dense_1, built=True>


In [41]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 32792, 64)         │      2,112 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ p_re_lu (PReLU)                 │ (None, 32792, 64)         │  2,098,688 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv1d_1 (Conv1D)               │ (None, 32777, 64)         │     65,600 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten (Flatten)               │ (None, 2097728)           │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 64)                │ 134,254,6… │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_1 (Dense)                 │ (None, 5)                 │        325 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 272,842,764 (1.02 GB)

 Trainable params: 136,421,381 (520.41 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 136,421,383 (520.41 MB)

In [52]:
test = model.predict(X[:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step


In [37]:
print(X[0])
print(X[0:1])

[   3    1    1 ... 5797 5875 5767]
[[   3    1    1 ... 5797 5875 5767]]


In [31]:
for i in range(100):
    # Format the list to display only the first two decimal places
    formatted_values = ["{:.2f}".format(value) for value in test[i]]

    # Print the formatted list
    print(formatted_values)
    print(Y[i])

    user_input = input("Enter 'quit' to exit, or any other input to continue: ")

    if user_input.lower() == 'quit':
        print("Exiting the loop.")
        break  # Exit the loop if the user enters 'quit'
    
    # Add your code here for each iteration
    print("Loop iteration. You entered:", user_input)

['0.95', '0.00', '0.04', '0.00', '0.00']
[0 0 0 0 1]
Loop iteration. You entered: 5
['0.79', '0.12', '0.08', '0.00', '0.00']
[0 0 0 0 1]
Loop iteration. You entered: 5
['0.97', '0.00', '0.03', '0.00', '0.00']
[0 0 0 0 1]
Loop iteration. You entered: 5
['0.98', '0.00', '0.01', '0.01', '0.00']
[0 0 0 0 1]
Loop iteration. You entered: 5
['0.86', '0.08', '0.02', '0.04', '0.00']
[0 0 0 0 1]
Loop iteration. You entered: 5


IndexError: index 5 is out of bounds for axis 0 with size 5